In [77]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

### Sparse Optical Flow

In [75]:
def capture_moving_sparse(prev_frame, prev_feat, frame, mask):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

    
    feat, status, error = cv2.calcOpticalFlowPyrLK(prev_frame_gray, frame_gray, prev_feat, None)

    for feature1, feature2 in zip(prev_feat, feat):
      f10=int(feature1[0][0])
      f11=int(feature1[0][1])
      f20=int(feature2[0][0])
      f21=int(feature2[0][1])
    
      # Choose treshold
      if(((f10-f20)**2+(f11-f21)**2)**(1/2)) > 2:
          cv2.line(mask, (f10,f11), (f20, f21), (0, 0, 255), 3)
          cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)
    
    return feat

In [76]:
cap = cv2.VideoCapture('Robots.mp4')

ret, prev_frame = cap.read()
mask = np.zeros_like(prev_frame)
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)
prev_feat = cv2.goodFeaturesToTrack(prev_frame_gray, maxCorners=100, qualityLevel=0.2, minDistance=5)

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    prev_feat = capture_moving_sparse(prev_frame, prev_feat, frame, mask)
    prev_frame = frame

    # Display the resulting frame
    cv2.imshow('Frame', cv2.add(frame, mask))

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

### Dense Optical Flow

In [79]:
cap = cv2.VideoCapture('Robots.mp4')

ret, prev_frame = cap.read()
mask = np.zeros_like(prev_frame)
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)
prev_feat = cv2.goodFeaturesToTrack(prev_frame_gray, maxCorners=100, qualityLevel=0.2, minDistance=5)

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

    flow = cv2.calcOpticalFlowFarneback(frame_gray, prev_frame_gray, None, 0.5, 3, 15, 3, 5, 1.5, 0)
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])

    for row, (mag_rows,ang_rows) in enumerate(zip(mag,ang)):
      for column, (magnitude, angle) in enumerate(zip(mag_rows, ang_rows)):
          
          if row % 10 == 0 and column % 10 == 0:
              magnitude *= 10
              arg = (column, row)
              # cv2.line(mask, (arg), (arg[0] + int(math.cos(angle)*magnitude), arg[1]+int(math.sin(angle)*magnitude)), (0, 0, 255), 1)
              cv2.circle(mask, (arg), 2, (255, 0, 0), -1)

    prev_frame = frame

    # Display the resulting frame
    cv2.imshow('Frame', cv2.add(frame, mask))

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()